In [ ]:
import sqlite3
import pandas
import seaborn
import matplotlib.pyplot as plt

# Setting up user ratings

In [ ]:
my_ratings = pandas.read_csv('ratings.csv')

In [ ]:
# Dropping less clean columns that have cleaner counterparts in downloadable datasets
my_ratings.drop(['URL', 'Genres', 'Release Date', 'Directors'], axis=1, inplace=True, errors='ignore')

# Renaming columns to match downloadable datasets
my_ratings.rename(columns={
  'Const': 'tconst',
  'Title': 'primaryTitle',
  'Your Rating': 'yourRating',
  'Date Rated': 'dateRated',
  'Title Type': 'titleType',
  'IMDb Rating': 'averageRating',
  'Num Votes': 'numVotes',
  'Runtime (mins)': 'runtimeMinutes'
}, inplace=True)

In [ ]:
# From now on, we work with these columns
print(my_ratings.columns)

# Dropping everything but movies
my_ratings = my_ratings[my_ratings['titleType'] == 'movie']
print("Number of ratings:", len(my_ratings))

# Analysis 0 - Ratings histogram

**Computing a baseline of sorts.**

This may highlight biases towards specific numbers on the 1 to 10 scale.

In [ ]:
seaborn.set(rc = {'figure.figsize': (10,5)})
plot = seaborn.countplot(my_ratings['yourRating'], palette='flare')
_ = plot.set_xlabel('Rating')
_ = plot.set_ylabel('Count')

# Analysis 1 - Most and least popular movies

In [ ]:
rfdb = sqlite3.connect('rackfocus_out.db')
titles = pandas.read_sql_query('select tconst, startYear as releaseYear from title_basics', rfdb)
rfdb.close()

## Most popular

In [ ]:
my_ratings.merge(titles, on='tconst').sort_values(by='numVotes', ascending=False).head(15)[['primaryTitle', 'releaseYear', 'numVotes']]

## Least popular

In [ ]:
my_ratings.merge(titles, on='tconst').sort_values(by='numVotes', ascending=True).head(15)[['primaryTitle', 'releaseYear', 'numVotes']]

## Clearing dataframes

In [ ]:
titles.iloc[0:0]

# Analysis 2 - Directors

In [ ]:
rfdb = sqlite3.connect('rackfocus_out.db')
names = pandas.read_sql_query('select nconst, primaryName from name_basics', rfdb)
title_director = pandas.read_sql_query('select tconst, nconst from title_principals where category="director"', rfdb)
rfdb.close()

print(names.columns)
print(title_director.columns)

## "Favorite" directors

In [ ]:
my_r_director = my_ratings.merge(title_director, on='tconst')

### Cumulative ratings (raw)

In [ ]:
my_r_d_cumscore = my_r_director.groupby('nconst')['yourRating'].sum().reset_index()
my_r_d_cumscore_results = my_r_d_cumscore.sort_values(by='yourRating', ascending=False).head(50).merge(names, on='nconst')
my_r_d_cumscore_results.rename(columns={
  'primaryName': 'Director',
  'yourRating': 'Cumulative Score'
}, inplace=True)

In [ ]:
seaborn.set(rc = {'figure.figsize': (20,5)})
plot = seaborn.barplot(data=my_r_d_cumscore_results, x='Director', y='Cumulative Score', palette='flare_r')
_ = plot.set_xticklabels(plot.get_xticklabels(), rotation=45, horizontalalignment='right')

### Mean ratings (refined)

In [ ]:
my_r_d_avgscore = my_r_director.groupby('nconst')['yourRating'].mean().reset_index()
my_r_d_avgscore_results = my_r_d_avgscore.sort_values(by='yourRating', ascending=False).head(50).merge(names, on='nconst')
my_r_d_avgscore_results.rename(columns={
  'primaryName': 'Director',
  'yourRating': 'Cumulative Score'
}, inplace=True)
my_r_d_avgscore_results[['Director', 'Cumulative Score']]

## Clearing dataframes

In [ ]:
names.iloc[0:0]
title_director.iloc[0:0]
my_r_director.iloc[0:0]
my_r_d_cumscore.iloc[0:0]
my_r_d_avgscore.iloc[0:0]
my_r_d_cumscore_results.iloc[0:0]
my_r_d_avgscore_results.iloc[0:0]

# Analysis 3 - Through the years

In [ ]:
rfdb = sqlite3.connect('rackfocus_out.db')
titles = pandas.read_sql_query('select tconst, startYear as releaseYear from title_basics where titleType="movie"', rfdb)
rfdb.close()

## Movie count by release year

In [ ]:
my_r_titles_release_year = my_ratings.merge(titles, on='tconst')
my_r_titles_release_year.rename(columns={
  'count': 'Count',
  'releaseYear': 'Release Year'
}, inplace=True)
seaborn.set(rc = {'figure.figsize': (20,5)})
plot = seaborn.countplot(my_r_titles_release_year['Release Year'], palette='flare')
_ = plot.set_xticklabels(plot.get_xticklabels(), rotation=90)

**Plotting all movie release years on IMDb**

Data could be unclean.

In [ ]:
seaborn.set(rc = {'figure.figsize': (30,5)})
plot = seaborn.countplot(titles['releaseYear'], palette='flare')
_ = plot.set_xticklabels(plot.get_xticklabels(), rotation=90)
_ = plot.set_xlabel('Runtime (mins)')

## Release year / rating heatmap

In [ ]:
my_r_titles = my_ratings.merge(titles, on='tconst')
my_r_titles.rename(columns={
  'yourRating': 'Your Rating',
  'releaseYear': 'Release Year'
}, inplace=True)
my_r_release_year = pandas.crosstab(my_r_titles['Your Rating'], my_r_titles['Release Year'])
seaborn.set(rc = {'figure.figsize': (20,5)})
plot = seaborn.heatmap(my_r_release_year, cbar_kws={'label': 'Count'}, cmap='flare')
plot.invert_yaxis()

## Clearing dataframes

In [ ]:
my_r_titles_release_year.iloc[0:0]
my_r_titles.iloc[0:0]
my_r_release_year.iloc[0:0]

# Analysis 4 - Ratings by runtime

In [ ]:
rfdb = sqlite3.connect('rackfocus_out.db')
titles = pandas.read_sql_query('select tconst, runtimeMinutes from title_basics where titleType="movie"', rfdb)
rfdb.close()

## Movie count by runtime

In [ ]:
seaborn.set(rc = {'figure.figsize': (20,5)})
plot = seaborn.countplot(my_ratings['runtimeMinutes'], palette='flare')
_ = plot.set_xticklabels(plot.get_xticklabels(), rotation=90)
_ = plot.set_xlabel('Runtime (mins)')

**Plotting all movie runtimes on IMDb**

I have no explanations for the patterns nor the extremes. Data could be unclean.

In [ ]:
seaborn.set(rc = {'figure.figsize': (90,5)})
plot = seaborn.countplot(titles['runtimeMinutes'], palette='flare')
_ = plot.set_xticklabels(plot.get_xticklabels(), rotation=90)
_ = plot.set_xlabel('Runtime (mins)')

## Runtime / rating heatmap

In [ ]:
my_r_runtime = pandas.crosstab(my_ratings['yourRating'], my_ratings['runtimeMinutes'])
seaborn.set(rc = {'figure.figsize': (20,5)})
plot = seaborn.heatmap(my_r_runtime, cbar_kws={'label': 'Count'}, cmap='flare')
plot.invert_yaxis()

## Clearing dataframes

In [ ]:
titles.iloc[0:0]
my_r_runtime.iloc[0:0]

# Analysis 5 - Biggest ratings delta

In [ ]:
rfdb = sqlite3.connect('rackfocus_out.db')
titles = pandas.read_sql_query('select tconst, startYear as releaseYear from title_basics where titleType="movie"', rfdb)
rfdb.close()

## "Overrated" movies

In [ ]:
my_r_imdb_r_with_delta = my_ratings[['primaryTitle', 'yourRating', 'averageRating']].copy()
my_r_imdb_r_with_delta['ratingDelta'] = my_r_imdb_r_with_delta.apply(lambda r: r['averageRating'] - r['yourRating'], axis=1)
my_r_imdb_r_with_delta = my_r_imdb_r_with_delta.sort_values(by='ratingDelta', ascending=False).head(50).sort_values(by='averageRating', ascending=False)

In [ ]:
plt.figure(figsize=(20,25))
plt.hlines(y=my_r_imdb_r_with_delta['primaryTitle'], xmin=my_r_imdb_r_with_delta['yourRating'], xmax=my_r_imdb_r_with_delta['averageRating'], color='grey')
plt.scatter(my_r_imdb_r_with_delta['yourRating'], my_r_imdb_r_with_delta['primaryTitle'], color='#f37651', s=150)
plt.scatter(my_r_imdb_r_with_delta['averageRating'], my_r_imdb_r_with_delta['primaryTitle'], color='#ad1759', s=150)
plt.xlim(1, 10)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
ax = plt.gca()
ax.invert_yaxis()

## "Underrated" movies

In [ ]:
my_r_imdb_r_with_delta = my_ratings[['primaryTitle', 'yourRating', 'averageRating']].copy()
my_r_imdb_r_with_delta['ratingDelta'] = my_r_imdb_r_with_delta.apply(lambda r: r['yourRating'] - r['averageRating'], axis=1)
my_r_imdb_r_with_delta = my_r_imdb_r_with_delta.sort_values(by='ratingDelta', ascending=False).head(50).sort_values(by='averageRating', ascending=False)

In [ ]:
plt.figure(figsize=(20,25))
plt.hlines(y=my_r_imdb_r_with_delta['primaryTitle'], xmin=my_r_imdb_r_with_delta['yourRating'], xmax=my_r_imdb_r_with_delta['averageRating'], color='grey')
plt.scatter(my_r_imdb_r_with_delta['yourRating'], my_r_imdb_r_with_delta['primaryTitle'], color='#f37651', s=150)
plt.scatter(my_r_imdb_r_with_delta['averageRating'], my_r_imdb_r_with_delta['primaryTitle'], color='#ad1759', s=150)
plt.xlim(1, 10)
plt.xticks(fontsize=16)
plt.yticks(fontsize=16)
ax = plt.gca()

In [ ]:
titles.iloc[0:0]
my_r_imdb_r_with_delta.iloc[0:0]